In [1]:
import os
import time
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score
import torch.nn as nn
import pytorch_lightning as pl
from tqdm import tqdm

In [2]:
# Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
EPOCHS = 5
IMG_SIZE = 32  # 32x32 grayscale images
DATASET_DIR = '/kaggle/input/lra-pathfinder-32/pathfinder32/curv_contour_length_14'

In [3]:
# Dataset Class
class PathfinderDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        try:
            image = Image.open(img_path).convert('L')  # Grayscale
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
            new_idx = (idx + 1) % len(self.data)
            return self[new_idx]

In [4]:
# Data Module
class PathfinderDataModule(pl.LightningDataModule):
    def __init__(self, dataset_dir, batch_size=BATCH_SIZE):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.Resize((IMG_SIZE, IMG_SIZE)),
            transforms.ToTensor(),
        ])

    def prepare_data(self):
        self.data_list = []
        metadata_dir = os.path.join(self.dataset_dir, 'metadata')
        for file_name in os.listdir(metadata_dir):
            metadata_path = os.path.join(metadata_dir, file_name)
            with open(metadata_path, 'r') as file:
                for line in file:
                    tokens = line.strip().split()
                    img_rel_path = tokens[0] + '/' + tokens[1]
                    label = int(tokens[3])
                    img_path = os.path.join(self.dataset_dir, img_rel_path)
                    self.data_list.append((img_path, label))

    def setup(self, stage=None):
        dataset = PathfinderDataset(self.data_list, transform=self.transform)
        train_size = int(0.8 * len(dataset))
        val_size = int(0.1 * len(dataset))
        test_size = len(dataset) - train_size - val_size
        self.train_set, self.val_set, self.test_set = random_split(dataset, [train_size, val_size, test_size])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, num_workers=4)

In [5]:
# Transformer Model
class SequenceTransformer(nn.Module):
    def __init__(self, img_size, num_classes):
        super().__init__()
        self.img_size = img_size
        self.num_patches = img_size * img_size
        self.patch_dim = 1
        self.embedding_dim = 64

        self.embedding = nn.Linear(self.patch_dim, self.embedding_dim)
        self.transformer = nn.Transformer(
            d_model=self.embedding_dim,
            nhead=4,
            num_encoder_layers=3
        )
        self.classifier = nn.Linear(self.embedding_dim, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.patch_dim)  # Flatten to sequence
        x = self.embedding(x)
        x = self.transformer(x, x)
        x = x.mean(dim=1)  # Global average pooling
        x = self.classifier(x)
        return x

In [6]:
# Initialize Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SequenceTransformer(img_size=IMG_SIZE, num_classes=2).to(device)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [7]:
# Training and Evaluation
def train_model(model, train_loader, optimizer, loss_fn, epochs):
    print("Starting training...")
    model.train()
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        print(f"Epoch {epoch + 1} Loss: {total_loss / len(train_loader):.4f}")

In [8]:
def evaluate_model(model, test_loader):
    print("Evaluating model...")
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=-1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy

In [9]:
data_module = PathfinderDataModule(DATASET_DIR)
data_module.prepare_data()
data_module.setup()

train_loader = data_module.train_dataloader()
test_loader = data_module.test_dataloader()

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

start_time = time.time()
train_model(model, train_loader, optimizer, loss_fn, EPOCHS)
train_time = time.time() - start_time

accuracy = evaluate_model(model, test_loader)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
efficiency = accuracy / (torch.log(torch.tensor(train_time)) * torch.log(torch.tensor(num_params)))

print("\nTraining Metrics:")
print(f"Time Taken: {train_time:.2f} seconds")
print(f"Accuracy: {accuracy:.4f}")
print(f"Number of Parameters: {num_params}")
print(f"Efficiency Score: {efficiency:.4f}")

Starting training...
Epoch 1/5


Training Epoch 1: 100%|██████████| 5000/5000 [33:45<00:00,  2.47it/s]


Epoch 1 Loss: 0.6945
Epoch 2/5


Training Epoch 2: 100%|██████████| 5000/5000 [33:49<00:00,  2.46it/s]


Epoch 2 Loss: 0.6937
Epoch 3/5


...

Epoch 4 Loss: 0.6934
Epoch 5/5


Training Epoch 5: 100%|██████████| 5000/5000 [33:46<00:00,  2.47it/s]


Epoch 5 Loss: 0.6934
Evaluating model...


Evaluating: 100%|██████████| 625/625 [01:13<00:00,  8.46it/s]


Test Accuracy: 0.5011

Training Metrics:
Time Taken: 10138.11 seconds
Accuracy: 0.5011
Number of Parameters: 2631490
Efficiency Score: 0.0037
